In [1]:
import os

import django
from django.core.management import call_command

# Simulate manage.py
os.environ["DJANGO_SETTINGS_MODULE"] = "arches_querysets.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

call_command("delete_test_data")
call_command("add_test_data")

Deleting test data...
Finished!
Creating test data...
Finished!


In [2]:
from arches_querysets.models import *

resource = ResourceTileTree.get_tiles(graph_slug="datatype_lookups").exclude(url=None).get()
print(resource.aliased_data)
print(resource.aliased_data.datatypes_1.aliased_data)

AliasedData(datatypes_1=<TileTree: datatypes_1 (ee6df8cb-2988-45c3-ab0f-b9bdee91b34e)>, datatypes_n=[<TileTree: datatypes_n (68f90a43-613a-401c-b565-bc4c473258bd)>])
AliasedData(string={'en': {'value': 'forty-two', 'direction': 'ltr'}}, number=42, concept=<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>, concept_list=[<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>], date='2042-04-02', node_value='ee6df8cb-2988-45c3-ab0f-b9bdee91b34e', edtf=None, annotation=None, url={'url': 'http://www.42.com/', 'url_label': '42.com'}, resource_instance=<ResourceInstance: Datatype Lookups: Resource referencing 42 (de9ac93c-2485-4355-b3d5-80faf0472e9c)>, resource_instance_list=[<ResourceInstance: Datatype Lookups: Resource referencing 42 (de9ac93c-2485-4355-b3d5-80faf0472e9c)>], boolean=True, domain_value=None, domain_value_list=None, non_localized_string='forty-two', geojson_feature_collection=None, file_list=[{'url': 'http://www.archesproject.org/blog/static/42.png', 'name': 

In [3]:
resource = ResourceTileTree.get_tiles(
    graph_slug="datatype_lookups", as_representation=True
).exclude(url=None).get()

from pprint import pprint
pprint(resource.aliased_data)
pprint(resource.aliased_data.datatypes_1.aliased_data)

AliasedData(datatypes_1=<TileTree: datatypes_1 (ee6df8cb-2988-45c3-ab0f-b9bdee91b34e)>,
            datatypes_n=[<TileTree: datatypes_n (68f90a43-613a-401c-b565-bc4c473258bd)>])
AliasedData(string={'display_value': 'forty-two',
                    'node_value': {'en': {'direction': 'ltr',
                                          'value': 'forty-two'}}},
            number={'display_value': '42', 'node_value': 42},
            concept={'details': {'concept_id': '00000000-0000-0000-0000-000000000001',
                                 'language_id': 'en',
                                 'value': 'Arches',
                                 'valueid': 'd8c60bf4-e786-11e6-905a-b756ec83dad5',
                                 'valuetype_id': 'prefLabel'},
                     'display_value': 'Arches',
                     'node_value': 'd8c60bf4-e786-11e6-905a-b756ec83dad5'},
            concept_list={'details': [{'concept_id': '00000000-0000-0000-0000-000000000001',
                        

In [4]:
test_resources = ResourceTileTree.get_tiles("datatype_lookups", as_representation=True)
# This "shallow" lookup works whether or not the node is nested (todo: add test case)
resource = test_resources.filter(string__any_lang_contains="forty").first()
resource.aliased_data.datatypes_1.aliased_data.string

{'node_value': {'en': {'value': 'forty-two', 'direction': 'ltr'}},
 'display_value': 'forty-two'}

In [ ]:
# You can save back with or without the display value / node_value wrapper
resource.aliased_data.datatypes_1.aliased_data.string = 'forty-three'
resource.save()
# But without a request, we assume anonymous user, and it ends up in provisional edits
resource.aliased_data.datatypes_1.provisionaledits

{'2': {'value': {'10c37b4a-58a2-4403-b5cb-e59f7664f0a5': None,
   '1bcb95e2-92b7-4688-9d6b-f07b0bea304b': None,
   '23e7d826-635a-43d1-8152-fce1ac9e8b69': {'en': {'value': 'forty-three',
     'direction': 'ltr'}},
   '31681ba4-c3f0-4a7a-922d-e033347e5dff': True,
   '74637e6f-bca4-4f41-b4b1-497f89f95def': [{'resourceId': 'de9ac93c-2485-4355-b3d5-80faf0472e9c',
     'ontologyProperty': '',
     'resourceXresourceId': '7467bb75-ec45-4e32-8dce-ba16850fa86d',
     'inverseOntologyProperty': ''}],
   '76bda89b-18f8-401b-88ce-e70433e49242': None,
   '8486491d-035d-44d7-a833-585f721456a0': 'forty-two',
   '88accd4c-d5da-49a6-976c-1c2fa896bb8d': None,
   '88f15b81-b48a-4e22-9593-81e8aeccf919': [{'url': '/files/805bb024-f946-4360-aa4e-1dbba443cfb6',
     'name': '42_accessibility_improvements.png',
     'type': 'image/png',
     'status': 'uploaded',
     'file_id': '805bb024-f946-4360-aa4e-1dbba443cfb6',
     'accepted': True,
     'renderer': '5e05aa2e-5db0-4922-8938-b4d2b7919733'}],
   '8eb31

In [6]:
resource.aliased_data.datatypes_1.aliased_data.string = 'forty-three'
resource.save(force_admin=True)
resource.aliased_data.datatypes_1.aliased_data.string

{'node_value': {'en': {'value': 'forty-three', 'direction': 'ltr'}},
 'display_value': 'forty-three'}

In [7]:
# Custom lookups
resources = ResourceTileTree.get_tiles(
    "datatype_lookups", as_representation=True
).filter(string__any_lang_contains="forty")
for match in resources:
    print(match)

Datatype Lookups: Resource referencing 42 (de9ac93c-2485-4355-b3d5-80faf0472e9c)


In [8]:
resources = ResourceTileTree.get_tiles(
    "datatype_lookups", as_representation=True
).filter(resource_instance_list__contains=str(resource.pk))
for match in resources:
    print(match)

Datatype Lookups: Resource referencing 42 (de9ac93c-2485-4355-b3d5-80faf0472e9c)
